In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layers in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [3]:
# 通常のscikit-learn回帰器と同じように扱える。
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [4]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)
X_new = X_test[:3]

keras_reg.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)] 
)

mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 2.0959 - val_loss: 0.6981
Epoch 2/100
363/363 [==============================] - 0s 413us/step - loss: 0.6990 - val_loss: 0.6017
Epoch 3/100
363/363 [==============================] - 0s 413us/step - loss: 0.6114 - val_loss: 0.5547
Epoch 4/100
363/363 [==============================] - 0s 411us/step - loss: 0.5956 - val_loss: 0.5119
Epoch 5/100
363/363 [==============================] - 0s 411us/step - loss: 0.5165 - val_loss: 0.4867
Epoch 6/100
363/363 [==============================] - 0s 412us/step - loss: 0.5259 - val_loss: 0.4654
Epoch 7/100
363/363 [==============================] - 0s 411us/step - loss: 0.4882 - val_loss: 0.4540
Epoch 8/100
363/363 [==============================] - 0s 411us/step - loss: 0.4982 - val_loss: 0.4427
Epoch 9/100
363/363 [==============================] - 0s 411us/step - loss: 0.4622 - val_loss: 0.4369
Epoch 10/100
363/363 [==============================] - 0s 417us/step - los

Epoch 80/100
363/363 [==============================] - 0s 407us/step - loss: 0.3737 - val_loss: 0.3434
Epoch 81/100
363/363 [==============================] - 0s 407us/step - loss: 0.3645 - val_loss: 0.3434
Epoch 82/100
363/363 [==============================] - 0s 406us/step - loss: 0.3675 - val_loss: 0.3455
Epoch 83/100
363/363 [==============================] - 0s 406us/step - loss: 0.3601 - val_loss: 0.3459
Epoch 84/100
363/363 [==============================] - 0s 407us/step - loss: 0.3566 - val_loss: 0.3435
Epoch 85/100
363/363 [==============================] - 0s 407us/step - loss: 0.3777 - val_loss: 0.3425
Epoch 86/100
363/363 [==============================] - 0s 407us/step - loss: 0.3757 - val_loss: 0.3409
Epoch 87/100
363/363 [==============================] - 0s 407us/step - loss: 0.3575 - val_loss: 0.3410
Epoch 88/100
363/363 [==============================] - 0s 407us/step - loss: 0.3629 - val_loss: 0.3407
Epoch 89/100
363/363 [==============================] - 0s 408us

In [5]:
# ランダムサーチ
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

Epoch 1/100
242/242 [==============================] - 0s 677us/step - loss: 4.7721 - val_loss: 2.1823
Epoch 2/100
242/242 [==============================] - 0s 434us/step - loss: 1.7275 - val_loss: 1.1057
Epoch 3/100
242/242 [==============================] - 0s 437us/step - loss: 1.0038 - val_loss: 0.8099
Epoch 4/100
242/242 [==============================] - 0s 435us/step - loss: 0.8016 - val_loss: 0.7183
Epoch 5/100
242/242 [==============================] - 0s 437us/step - loss: 0.6989 - val_loss: 0.6804
Epoch 6/100
242/242 [==============================] - 0s 437us/step - loss: 0.6945 - val_loss: 0.6597
Epoch 7/100
242/242 [==============================] - 0s 433us/step - loss: 0.6940 - val_loss: 0.6441
Epoch 8/100
242/242 [==============================] - 0s 431us/step - loss: 0.6661 - val_loss: 0.6325
Epoch 9/100
242/242 [==============================] - 0s 435us/step - loss: 0.6517 - val_loss: 0.6205
Epoch 10/100
242/242 [==============================] - 0s 435us/step - l

Epoch 80/100
242/242 [==============================] - 0s 463us/step - loss: 0.5358 - val_loss: 0.5195
Epoch 81/100
121/121 [==============================] - 0s 244us/step - loss: 0.5502
Epoch 1/100
242/242 [==============================] - 0s 646us/step - loss: 5.6154 - val_loss: 2.2685
Epoch 2/100
242/242 [==============================] - 0s 435us/step - loss: 1.7464 - val_loss: 1.0782
Epoch 3/100
242/242 [==============================] - 0s 433us/step - loss: 0.9528 - val_loss: 0.7615
Epoch 4/100
242/242 [==============================] - 0s 432us/step - loss: 0.7498 - val_loss: 0.6637
Epoch 5/100
242/242 [==============================] - 0s 434us/step - loss: 0.6506 - val_loss: 0.6254
Epoch 6/100
242/242 [==============================] - 0s 433us/step - loss: 0.6444 - val_loss: 0.6104
Epoch 7/100
242/242 [==============================] - 0s 430us/step - loss: 0.5938 - val_loss: 0.6016
Epoch 8/100
242/242 [==============================] - 0s 436us/step - loss: 0.6419 - val_

242/242 [==============================] - 0s 431us/step - loss: 0.5391 - val_loss: 0.5108
Epoch 78/100
242/242 [==============================] - 0s 434us/step - loss: 0.5526 - val_loss: 0.5174
Epoch 79/100
242/242 [==============================] - 0s 433us/step - loss: 0.5342 - val_loss: 0.5225
Epoch 80/100
242/242 [==============================] - 0s 434us/step - loss: 0.5476 - val_loss: 0.5129
Epoch 81/100
121/121 [==============================] - 0s 240us/step - loss: 0.5251
Epoch 1/100
242/242 [==============================] - 0s 637us/step - loss: 6.8020 - val_loss: 2.7488
Epoch 2/100
242/242 [==============================] - 0s 436us/step - loss: 1.9979 - val_loss: 1.1531
Epoch 3/100
242/242 [==============================] - 0s 437us/step - loss: 0.9870 - val_loss: 0.7441
Epoch 4/100
242/242 [==============================] - 0s 436us/step - loss: 0.7054 - val_loss: 0.6226
Epoch 5/100
242/242 [==============================] - 0s 435us/step - loss: 0.6168 - val_loss: 0.58

242/242 [==============================] - 0s 462us/step - loss: 0.4802 - val_loss: 0.4403
Epoch 5/100
242/242 [==============================] - 0s 462us/step - loss: 0.4611 - val_loss: 0.4388
Epoch 6/100
242/242 [==============================] - 0s 462us/step - loss: 0.4422 - val_loss: 0.6767
Epoch 7/100
242/242 [==============================] - 0s 461us/step - loss: 0.4476 - val_loss: 0.4281
Epoch 8/100
242/242 [==============================] - 0s 463us/step - loss: 0.4455 - val_loss: 0.3967
Epoch 9/100
242/242 [==============================] - 0s 463us/step - loss: 0.4312 - val_loss: 0.4023
Epoch 10/100
242/242 [==============================] - 0s 461us/step - loss: 0.4415 - val_loss: 0.3984
Epoch 11/100
242/242 [==============================] - 0s 462us/step - loss: 0.4035 - val_loss: 0.3955
Epoch 12/100
242/242 [==============================] - 0s 462us/step - loss: 0.4164 - val_loss: 0.3936
Epoch 13/100
242/242 [==============================] - 0s 462us/step - loss: 0.40

121/121 [==============================] - 0s 238us/step - loss: 0.3578
Epoch 1/100
242/242 [==============================] - 0s 690us/step - loss: 1.3656 - val_loss: 0.6271
Epoch 2/100
242/242 [==============================] - 0s 464us/step - loss: nan - val_loss: nan
Epoch 3/100
242/242 [==============================] - 0s 462us/step - loss: nan - val_loss: nan
Epoch 4/100
242/242 [==============================] - 0s 464us/step - loss: nan - val_loss: nan
Epoch 5/100
242/242 [==============================] - 0s 463us/step - loss: nan - val_loss: nan
Epoch 6/100
242/242 [==============================] - 0s 462us/step - loss: nan - val_loss: nan
Epoch 7/100
242/242 [==============================] - 0s 465us/step - loss: nan - val_loss: nan
Epoch 8/100
242/242 [==============================] - 0s 461us/step - loss: nan - val_loss: nan
Epoch 9/100
242/242 [==============================] - 0s 464us/step - loss: nan - val_loss: nan
Epoch 10/100
242/242 [===========================

...

121/121 [==============================] - 0s 235us/step - loss: 0.4809
Epoch 1/100
242/242 [==============================] - 0s 678us/step - loss: 5.1876 - val_loss: 2.8132
Epoch 2/100
242/242 [==============================] - 0s 449us/step - loss: 2.4828 - val_loss: 1.6679
Epoch 3/100
242/242 [==============================] - 0s 450us/step - loss: 1.5689 - val_loss: 1.2150
Epoch 4/100
242/242 [==============================] - 0s 451us/step - loss: 1.1393 - val_loss: 0.9996
Epoch 5/100
242/242 [==============================] - 0s 455us/step - loss: 0.9718 - val_loss: 0.8822
Epoch 6/100
242/242 [==============================] - 0s 452us/step - loss: 0.8608 - val_loss: 0.8124
Epoch 7/100
242/242 [==============================] - 0s 451us/step - loss: 0.8120 - val_loss: 0.7679
Epoch 8/100
242/242 [==============================] - 0s 453us/step - loss: 0.7685 - val_loss: 0.7380
Epoch 9/100
242/242 [==============================] - 0s 448us/step - loss: 0.7735 - val_loss: 0.7169
E

242/242 [==============================] - 0s 463us/step - loss: 0.5019 - val_loss: 0.4670
Epoch 80/100
242/242 [==============================] - 0s 462us/step - loss: 0.5096 - val_loss: 0.4661
Epoch 81/100
242/242 [==============================] - 0s 468us/step - loss: 0.4833 - val_loss: 0.4651
Epoch 82/100
242/242 [==============================] - 0s 464us/step - loss: 0.5111 - val_loss: 0.4641
Epoch 83/100
242/242 [==============================] - 0s 466us/step - loss: 0.4896 - val_loss: 0.4632
Epoch 84/100
242/242 [==============================] - 0s 464us/step - loss: 0.4706 - val_loss: 0.4623
Epoch 85/100
242/242 [==============================] - 0s 467us/step - loss: 0.4848 - val_loss: 0.4614
Epoch 86/100
242/242 [==============================] - 0s 466us/step - loss: 0.4849 - val_loss: 0.4606
Epoch 87/100
242/242 [==============================] - 0s 466us/step - loss: 0.4807 - val_loss: 0.4597
Epoch 88/100
242/242 [==============================] - 0s 464us/step - loss:

242/242 [==============================] - 0s 503us/step - loss: 0.4896 - val_loss: 0.4930
Epoch 58/100
242/242 [==============================] - 0s 450us/step - loss: 0.5275 - val_loss: 0.4912
Epoch 59/100
242/242 [==============================] - 0s 458us/step - loss: 0.4997 - val_loss: 0.4895
Epoch 60/100
242/242 [==============================] - 0s 455us/step - loss: 0.5071 - val_loss: 0.4878
Epoch 61/100
242/242 [==============================] - 0s 456us/step - loss: 0.5098 - val_loss: 0.4862
Epoch 62/100
242/242 [==============================] - 0s 460us/step - loss: 0.5103 - val_loss: 0.4847
Epoch 63/100
242/242 [==============================] - 0s 460us/step - loss: 0.5184 - val_loss: 0.4831
Epoch 64/100
242/242 [==============================] - 0s 458us/step - loss: 0.5019 - val_loss: 0.4816
Epoch 65/100
242/242 [==============================] - 0s 458us/step - loss: 0.5011 - val_loss: 0.4800
Epoch 66/100
242/242 [==============================] - 0s 463us/step - loss:

/opt/anaconda3/envs/local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [-0.54290271         nan -0.32331023 -0.54020707 -0.31799061 -0.31490263
 -0.48334331 -0.40989488 -0.43331728 -0.47894455]
  warnings.warn(


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f88101be400>, as the constructor either does not set or modifies parameter learning_rate

In [ ]:
rnd_search_cv.best_params_

In [ ]:
rnd_search_cv.best_score_